# Setup

In [131]:
import sys
!{sys.executable} -m pip install 'pandas~=1.0.5'
!{sys.executable} -m pip install 'cufflinks'
!{sys.executable} -m pip install 'textdistance[extras]'
!{sys.executable} -m pip install 'chart_studio'

  Using cached https://files.pythonhosted.org/packages/5d/24/91ad2da4a1da2747595d2f47f858a131036f598fb495ec6346d08d8b8df6/pandas-1.0.5-cp37-cp37m-macosx_10_9_x86_64.whl
  Found existing installation: pandas 0.23.4
    Uninstalling pandas-0.23.4:
      Successfully uninstalled pandas-0.23.4


In [1]:
import os
from os.path import dirname, abspath
import sys
CRAWLER_DIR = dirname(os.path.abspath('')) + "/columbia_crawler"
sys.path.insert(0, CRAWLER_DIR)
from columbia_crawler import config

import pandas as pd
from pandas.io.json import json_normalize

import numpy as np

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import textdistance

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/soid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Wiki Search Results Classifier

In [10]:
from importlib import reload
import wiki_search_train
reload(wiki_search_train)

model = wiki_search_train.WikiSearchClassifier()

In [11]:
model.load_training_data()
data = model.data

Initial data for training before balancing:


0    727
1     50
2     31
Name: search_results.label, dtype: int64

Train set:


1    25
0    25
2    24
Name: search_results.label, dtype: int64

Test set:


2    7
1    6
0    6
Name: search_results.label, dtype: int64

In [12]:
model.fit()
model.evaluate()

Evaluating test data set:
  Misclassified as irrelevant but are relevant (no big deal): [5, 7, 8, 14]

              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       0.56      0.83      0.67         6
           2       1.00      0.43      0.60         7

    accuracy                           0.74        19
   macro avg       0.80      0.75      0.73        19
weighted avg       0.81      0.74      0.72        19

Predicted:
[2 0 0 0 1 1 0 1 1 1 0 1 1 2 1 0 2 1 0]
True:
[2 0 0 0 1 2 0 2 2 1 1 1 1 2 2 0 2 1 0]

Evaluating the entire unbalanced data set:
  Misclassified as irrelevant but are relevant (no big deal): [2, 60, 405, 748, 760, 770]
  Misclassified as relevant but are not (big deal, unless mistake in dataset): [252, 543]

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       727
           1       0.61      0.92      0.74        50
           2       0.93      0.

# Experimental Models

In [817]:
# experiment: NB on separate labels into two models
y1_train = y_train.apply(lambda x: 1 if x == 1 else 0)
y2_train = y_train.apply(lambda x: 1 if x == 2 else 0)
y1_test = y_test.apply(lambda x: 1 if x == 1 else 0)
y2_test = y_test.apply(lambda x: 1 if x == 2 else 0)

print("label = 1")
clf = GaussianNB()
clf.fit(X_train, y1_train)
ypred = clf.predict(X_test)
eval_results(y1_test, ypred)
print()

print("label = 2")
clf = GaussianNB()
clf.fit(X_train, y2_train)
ypred = clf.predict(X_test)
eval_results(y2_test, ypred)

label = 1
              precision    recall  f1-score   support

           0       0.82      0.75      0.78        12
           1       0.57      0.67      0.62         6

    accuracy                           0.72        18
   macro avg       0.69      0.71      0.70        18
weighted avg       0.74      0.72      0.73        18

predicted:
[0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0]
true:
[0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1]


search_results.label     
2                     494    0
0                     730    0
1                     53     1
                      362    1
2                     299    0
1                     656    1
0                     394    0
2                     749    0
                      760    0
0                     647    0
                      258    0
                      652    0
1                     130    1
2                     748    0
                      405    0
1                     129    1
0                     318    0
1                     0      1
Name: search_results.label, dtype: int64


label = 2
              precision    recall  f1-score   support

           0       0.75      1.00      0.86        12
           1       1.00      0.33      0.50         6

    accuracy                           0.78        18
   macro avg       0.88      0.67      0.68        18
weighted avg       0.83      0.78      0.74        18

predicted:
[0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
true:
[1 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0]


search_results.label     
2                     494    1
0                     730    0
1                     53     0
                      362    0
2                     299    1
1                     656    0
0                     394    0
2                     749    1
                      760    1
0                     647    0
                      258    0
                      652    0
1                     130    0
2                     748    1
                      405    1
1                     129    0
0                     318    0
1                     0      0
Name: search_results.label, dtype: int64

In [818]:
# Train multi-class model
clf = GaussianNB()
clf.fit(X_train, y_train)

# Evaluate model
ypred = clf.predict(X_test)
eval_results(y_test, ypred)

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         6
           1       0.50      0.67      0.57         6
           2       0.67      0.67      0.67         6

    accuracy                           0.67        18
   macro avg       0.72      0.67      0.68        18
weighted avg       0.72      0.67      0.68        18

predicted:
[2 0 1 1 2 2 0 2 1 0 1 1 1 1 2 1 0 2]
true:
[2 0 1 1 2 1 0 2 2 0 0 0 1 2 2 1 0 1]


search_results.label     
2                     494    2
0                     730    0
1                     53     1
                      362    1
2                     299    2
1                     656    1
0                     394    0
2                     749    2
                      760    2
0                     647    0
                      258    0
                      652    0
1                     130    1
2                     748    2
                      405    2
1                     129    1
0                     318    0
1                     0      1
Name: search_results.label, dtype: int64

# Precision/Recall Trade-off Calculation

In [22]:
# taken from https://www.kaggle.com/kevinarvai/fine-tuning-a-classifier-in-scikit-learn

from sklearn.metrics import precision_recall_curve
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier

import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot


def test():
    clf_o = OneVsRestClassifier(RandomForestClassifier())
    
    y_train_bins = preprocessing.label_binarize(model.y_train, classes=[0, 1, 2])
    clf_o.fit(model.X_train, y_train_bins)
    
    X = list(data.apply(lambda x: model.extract_features2vector(x.to_dict()), axis=1))
    y = data['search_results.label']
    
    
    y_scores = clf_o.predict_proba(X)
    y_bins = preprocessing.label_binarize(y, classes=[0, 1, 2])
    
    cufflinks.go_offline()
    cufflinks.set_config_file(world_readable=True, theme='pearl')

    for i in range(3):
        precision_recall = precision_recall_curve(y_bins[:, i], y_scores[:, i])
        precision_recall = pd.DataFrame.from_records(precision_recall).T
        precision_recall.columns = ['Precision', 'Recall', 'Thresholds']
        precision_recall.iplot(x='Thresholds', xTitle='Decision Threshold',yTitle='Score', 
                               title='Precision/Recall Tradeoff for label: ' + str(i))

test()

# Draft

In [15]:
# Viewing samples and its features
model.view_sample(2)
model.view_sample(188)
model.view_sample(254)
model.view_sample(237)
model.view_sample(462)

{'search_results.title': 'Delphine Fawundu', 'search_results.snippet': 'Adama Delphine Fawundu (born 1971) is an American multi-disciplinary photographer and visual artist promoting African culture and heritage, a co-founder', 'search_results.label': 2, 'name': 'Delphine A Fawundu', 'department': 'Visual Arts'}
{'__name_match_title': True, '__name_match_title2': True, '__name_similarity_title': 0.9652777777777778, '__name_match_snippet': True, '__name_match_snippet2': True, '__department_similarity_snippet': 0.5249601275917065, '__columbia_match_snippet': False, '__word_adama': True, '__word_delphin': True, '__word_fawundu': True, '__word_(': True, '__word_born': True, '__word_1971': True, '__word_)': True, '__word_is': True, '__word_an': True, '__word_american': True, '__word_multi-disciplinari': True, '__word_photograph': True, '__word_and': True, '__word_visual': True, '__word_artist': True, '__word_promot': True, '__word_african': True, '__word_cultur': True, '__word_heritag': True

,search_results.title,search_results.snippet,search_results.label,name,department
2,Delphine Fawundu,Adama Delphine Fawundu (born 1971) is an Ameri...,2,Delphine A Fawundu,Visual Arts


{'search_results.title': 'Mark N. Brown', 'search_results.snippet': 'Mark Neil Brown (born November 18, 1951) is an American engineer, retired colonel in the United States Air Force and former NASA astronaut. Brown spent', 'search_results.label': 1, 'name': 'Mark Brown', 'department': 'Statistics'}
{'__name_match_title': True, '__name_match_title2': True, '__name_similarity_title': 0.9138461538461539, '__name_match_snippet': True, '__name_match_snippet2': True, '__department_similarity_snippet': 0.3799116997792493, '__columbia_match_snippet': False, '__word_mark': True, '__word_neil': True, '__word_brown': True, '__word_(': True, '__word_born': True, '__word_novemb': True, '__word_18': True, '__word_,': True, '__word_1951': True, '__word_)': True, '__word_is': True, '__word_an': True, '__word_american': True, '__word_engin': True, '__word_retir': True, '__word_colonel': True, '__word_in': True, '__word_the': True, '__word_unit': True, '__word_state': True, '__word_air': True, '__word_f

,search_results.title,search_results.snippet,search_results.label,name,department
188,Mark N. Brown,"Mark Neil Brown (born November 18, 1951) is an...",1,Mark Brown,Statistics


{'search_results.title': 'David Rosner', 'search_results.snippet': 'David Rosner is the Ronald H. Lauterstein Professor of Sociomedical Sciences and professor of history in the Graduate School of Arts and Sciences at Columbia', 'search_results.label': 2, 'name': 'David Rosner', 'department': 'Sociomedical Sciences'}
{'__name_match_title': True, '__name_match_title2': True, '__name_similarity_title': 1.0, '__name_match_snippet': True, '__name_match_snippet2': True, '__department_similarity_snippet': 0.5842685269436861, '__columbia_match_snippet': True, '__word_david': True, '__word_rosner': True, '__word_is': True, '__word_the': True, '__word_ronald': True, '__word_H.': True, '__word_lauterstein': True, '__word_professor': True, '__word_of': True, '__word_sociomed': True, '__word_scienc': True, '__word_and': True, '__word_histori': True, '__word_in': True, '__word_graduat': True, '__word_school': True, '__word_art': True, '__word_at': True, '__word_columbia': True}


,search_results.title,search_results.snippet,search_results.label,name,department
254,David Rosner,David Rosner is the Ronald H. Lauterstein Prof...,2,David Rosner,Sociomedical Sciences


{'search_results.title': 'Andrew Jacobs (journalist)', 'search_results.snippet': 'Andrew Jacobs is an American correspondent for The New York Times. Jacobs has been based in Beijing, China, since April 2008, covering the country for', 'search_results.label': 1, 'name': 'Andrew Jacobs', 'department': 'Finance'}
{'__name_match_title': True, '__name_match_title2': True, '__name_similarity_title': 0.9, '__name_match_snippet': True, '__name_match_snippet2': True, '__department_similarity_snippet': 0.5393650793650794, '__columbia_match_snippet': False, '__word_andrew': True, '__word_jacob': True, '__word_is': True, '__word_an': True, '__word_american': True, '__word_correspond': True, '__word_for': True, '__word_the': True, '__word_new': True, '__word_york': True, '__word_time': True, '__word_.': True, '__word_ha': True, '__word_been': True, '__word_base': True, '__word_in': True, '__word_beij': True, '__word_,': True, '__word_china': True, '__word_sinc': True, '__word_april': True, '__word_

,search_results.title,search_results.snippet,search_results.label,name,department
237,Andrew Jacobs (journalist),Andrew Jacobs is an American correspondent for...,1,Andrew Jacobs,Finance


{'search_results.title': 'Michael Eric Dyson', 'search_results.snippet': 'Michael Eric Dyson (born 1958) is an academic, author, preacher, and radio host. He is Professor of Sociology at Georgetown University. Described by Michael', 'search_results.label': 0, 'name': 'Michael P McNeil', 'department': 'Sociomedical Sciences'}
{'__name_match_title': False, '__name_match_title2': False, '__name_similarity_title': 0.8761752136752138, '__name_match_snippet': False, '__name_match_snippet2': False, '__department_similarity_snippet': 0.5845543345543346, '__columbia_match_snippet': False, '__word_michael': True, '__word_eric': True, '__word_dyson': True, '__word_(': True, '__word_born': True, '__word_1958': True, '__word_)': True, '__word_is': True, '__word_an': True, '__word_academ': True, '__word_,': True, '__word_author': True, '__word_preacher': True, '__word_and': True, '__word_radio': True, '__word_host': True, '__word_.': True, '__word_He': True, '__word_professor': True, '__word_of': Tr

,search_results.title,search_results.snippet,search_results.label,name,department
462,Michael Eric Dyson,"Michael Eric Dyson (born 1958) is an academic,...",0,Michael P McNeil,Sociomedical Sciences


In [54]:
# testing NB on small data
def test():
    X = np.array([{"a": True, "b": True}, {"a": True, "c": True}, {"b": True, "d": True}])
    Y = np.array([1, 1, 2])
    # Y = np.array([[1, 0], [1, 0], [0, 1]])

    d2v = get_dict2vec_converter(X)

    vecX = []
    for x in X:
        vecX.append(d2v(x))
    vecX = np.array(vecX)

    clf = GaussianNB()
    clf.fit(vecX, Y)
    test = d2v({'b': True})
    print(test)
    print(clf.predict([test]))
test()

[False, True, False, False]
[2]


In [16]:
# evaluating string distance/similarity functions
def test_distances(name, words_match_partial):
    print("name: ", name)
    print(words_match_partial("Delphine Fawundu", "Delphine A Fawundu"))
    print(words_match_partial("Mark Brown", "Mark N. Brown"))
    print(words_match_partial("Mark Brown", "Grisha Temchenko"))
    print(words_match_partial("Delphine Fawundu", "Delphine Fawundu"))
    print(words_match_partial("Andrew Jacobs (journalist)", "Andrew Jacobs"))
    print(words_match_partial("Andrew Jacobs", "Andrew Jacobs (journalist)"))
    print(words_match_partial('Michael Eric Dyson (born 1958) is an academic, author, preacher, and radio host. He is Professor of Sociology at Georgetown University. Described by Michael',
                             'Michael P McNeil'))
    print(words_match_partial('Christine Ann Wellington OBE (born 18 February 1977) is an English former professional triathlete and four-time Ironman Triathlon World Champion. She',
                             'Social Work'))
    print(words_match_partial('Adama Delphine Fawundu (born 1971) is an American multi-disciplinary photographer and visual artist promoting African culture and heritage, a co-founder',
                             'Visual Arts'))
    
test_distances("levenshtein", textdistance.levenshtein.normalized_similarity)
test_distances("jaro_winkler", textdistance.jaro_winkler.normalized_similarity)
test_distances("jaccard", lambda x, y: textdistance.jaccard.normalized_similarity(x.lower().split(), y.lower().split()))
test_distances("sorensen", lambda x, y: textdistance.sorensen.normalized_similarity(x.lower().split(), y.lower().split()))

name:  levenshtein
0.8888888888888888
0.7692307692307692
0.125
1.0
0.5
0.5
0.08974358974358976
0.05369127516778527
0.05921052631578949
name:  jaro_winkler
0.9777777777777777
0.9538461538461538
0.46666666666666656
1.0
0.9
0.9
0.5811965811965812
0.4686546674801708
0.5249601275917065
name:  jaccard
0.6666666666666666
0.6666666666666666
0.0
1.0
0.6666666666666666
0.6666666666666666
0.038461538461538436
0.0
0.04761904761904767
name:  sorensen
0.8
0.8
0.0
1.0
0.8
0.8
0.07407407407407407
0.0
0.09090909090909094


In [19]:
from sklearn.metrics import confusion_matrix

# Random forest classifier
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100)#, max_depth=3)
clf.fit(model.X_train, model.y_train)

# evaluate model
ypred = clf.predict(model.X_test)
print(classification_report(model.y_test, ypred))
model.show_true_pred(model.y_test, ypred)
display(model.y_test)

print("Confusion Matrix:")
print(pd.DataFrame(confusion_matrix(model.y_test, ypred),
        columns=['neg', 'possible', 'match'], index=['neg', 'possible', 'match']))

              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      0.83      0.91         6
           2       1.00      1.00      1.00         7

    accuracy                           0.95        19
   macro avg       0.95      0.94      0.94        19
weighted avg       0.95      0.95      0.95        19

Predicted:
[2 0 0 0 1 2 0 2 2 1 0 1 1 2 2 0 2 1 0]
True:
[2 0 0 0 1 2 0 2 2 1 1 1 1 2 2 0 2 1 0]


search_results.label    
2                     7     2
0                     14    0
                      20    0
                      7     0
1                     14    1
2                     28    2
0                     6     0
2                     6     2
                      13    2
1                     15    1
                      26    1
                      29    1
                      27    1
2                     14    2
                      2     2
0                     10    0
2                     11    2
1                     9     1
0                     28    0
Name: search_results.label, dtype: int64

Confusion Matrix:
          neg  possible  match
neg         6         0      0
possible    1         5      0
match       0         0      7
